In [1]:
import mapping_noRot as mappNR
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import SimpleITK as sitk
import time
import os
import ReadRawMHD as rR
t1 = time.time()

In [2]:
sample_list = ['S130684_L2_left', 'S130684_L2_right',
               'S131318_L1_left', 'S131318_L1_right', 'S131318_L2_left', 'S131318_L2_right',
               'S131318_L3_left', 'S131318_L3_right', 'S131318_L4_left', 'S131318_L4_right',
               'S131318_L5_left', 'S131318_L5_right',
               'S131788_L1_left', 'S131788_L1_right', 'S131788_L2_left', 'S131788_L2_right',
               'S131788_L3_left', 'S131788_L3_right', 'S131788_L4_left', 'S131788_L4_right',
               'S131835_L1_left', 'S131835_L1_right', 'S131835_L2_left', 'S131835_L2_right',
               'S131835_L3_left', 'S131835_L3_right', 'S131835_L4_left', 'S131835_L4_right',
               'S131840_L1_left', 'S131840_L1_right', 'S131840_L2_left', 'S131840_L2_right',
               'S131840_L3_left', 'S131840_L3_right']

In [3]:
sample_code = sample_list[2]
path_project = '/home/biomech/Documents/01_Icotec/'  # General project folder
path_ct = path_project + '01_Experiments/02_Scans/' + sample_code + '/04_Registered/'  # Folder of CT dat
file_bone = [filename for filename in os.listdir(path_ct + '/') if filename.endswith('image.mhd') and str(sample_code) in filename][0]
file = path_ct + file_bone
res = 0.0606995

In [4]:
# Area to evaluate
r_mm = 6  # radius in mm
r = int(np.rint(r_mm / res))
length = np.rint(np.array([-45, 0]) / res).astype(int)
drill = int(1.4/res)  # radius drill
print(min(length))
print(max(length))
print(r)

-741
0
99


In [5]:
bone_grey = sitk.ReadImage(file)
bone_img = np.transpose(sitk.GetArrayFromImage(bone_grey), [2, 1, 0])
bone_bvtv = rR.zeros_and_ones(bone_img, 320)
check_image = rR.zeros_and_ones(bone_img, 320)*2
ori = abs((np.array(bone_grey.GetOrigin())/res).astype(int))
print(ori)
print(ori[2]-741)

[312 531 862]
121


In [6]:
b=0
o=0
i=0
j=0
for z in range(min(length), max(length)):
    for y in range(-r, r):
        for x in range(-r, r):
            check_image[x+ori[0], y+ori[1], z+ori[2]] = 1
            if r**2 >= x**2 + y**2 > drill**2:
                i = i+1
                if bone_bvtv[x+ori[0], y+ori[1], z+ori[2]] == 1:
                    b = b+1
                elif bone_bvtv[x+ori[0], y+ori[1], z+ori[2]] == 0:
                    o = o+1
                else:
                    print('**ERROR**')
            else:
                j = j+1
print('BV/BV: ' + str(round(b/(b+o),3)))

BV/BV: 0.144


In [8]:
tRun = time.time() - t1
if tRun >= 3600:
    print('Execution time: ' + str(int(tRun / 3600)) + ' h ' + str(int(np.mod(tRun, 3600) / 60)) + ' min ' +
          str(round(np.mod(tRun, 60), 1)) + ' sec.')
elif tRun >= 60:
    print('Execution time: ' + str(int(tRun / 60)) + ' min ' + str(round(np.mod(tRun, 60), 1)) + ' sec.')
else:
    print('Execution time: ' + str(round(tRun, 1)) + ' sec.')

Execution time: 53.9 sec.
